<a href="https://colab.research.google.com/github/AnthonyBliss1/AnthonyBliss1.github.io/blob/main/seq_nn_UFC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
# Load the datasets
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')

# Define features and target
feature_columns = [
    'Fighter_Height', 'Fighter_Reach', 'Cumulative_Fighter_KD', 'Cumulative_Fighter_Sub_Att',
    'Cumulative_Fighter_REV', 'Opponent_Height', 'Opponent_Reach', 'Cumulative_Opponent_KD', 'Cumulative_Opponent_Sub_Att',
    'Cumulative_Opponent_REV','Fighter_Age_at_Fight',
    'Opponent_Age_at_Fight', 'Cumulative_Fighter_Sig_Strikes_Accuracy', 'Cumulative_Fighter_Total_Strikes_Accuracy',
    'Cumulative_Fighter_TD_Accuracy', 'Cumulative_Opponent_Sig_Strikes_Accuracy', 'Cumulative_Opponent_Total_Strikes_Accuracy',
    'Cumulative_Opponent_TD_Accuracy', 'Fighter_Form', 'Opponent_Form', 'Fighter_Streak', 'Opponent_Streak',
    'Fighter_Avg_Sig_Strikes_Landed',	'Fighter_Avg_Total_Strikes_Landed',	'Fighter_Avg_TD_Landed',	'Fighter_Avg_CNTRL_Time',
    'Opponent_Avg_Sig_Strikes_Landed',	'Opponent_Avg_Total_Strikes_Landed',	'Opponent_Avg_TD_Landed', 'Opponent_Avg_CNTRL_Time',
    'Diff_Avg_Sig_Strikes_Landed',	'Diff_Avg_Total_Strikes_Landed',	'Diff_Avg_TD_Landed',	'Diff_Avg_CNTRL_Time', 'Fighter_Win_Percentage', 'Opponent_Win_Percentage',
    'Fighter_Total_CNTRL_Minutes', 'Opponent_Total_CNTRL_Minutes','Cumulative_Fighter_No_Contest',	'Cumulative_Fighter_Draws', 'Cumulative_Opponent_No_Contest',	'Cumulative_Fighter_Draws', 'Cumulative_Fighter_Wins',	'Cumulative_Fighter_Losses', 'Cumulative_Opponent_Wins',	'Cumulative_Opponent_Losses'
]

# Prepare the feature data
X_train = train_df[feature_columns].values
X_test = test_df[feature_columns].values

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [11]:
# Check for NaN values in the dataset
if np.isnan(X_train).any() or np.isnan(X_test).any():
    print("NaN values detected in dataset.")
    X_train = np.nan_to_num(X_train)
    X_test = np.nan_to_num(X_test)

# One-hot encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(train_df['Fighter_Result'])
y_test_encoded = label_encoder.transform(test_df['Fighter_Result'])

y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

# Neural network architecture
model = Sequential()
model.add(Dense(64, input_dim=len(feature_columns), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))

# Adjusting the output layer for multi-class classification
model.add(Dense(y_train_categorical.shape[1], activation='softmax'))

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train_categorical, epochs=100, batch_size=32, verbose=1, validation_data=(X_test, y_test_categorical))

# Evaluate the model
scores = model.evaluate(X_test, y_test_categorical, verbose=0)
print('Test accuracy:', scores[1])

# Save the trained model
model.save('my_trained_model.h5')

# It's also a good practice to save the scaler
import joblib
joblib.dump(scaler, 'my_scaler.pkl')

NaN values detected in dataset.
Epoch 1/100
317/317 [==============================] - 2s 5ms/step - loss: 1.4554 - accuracy: 0.3209 - val_loss: 1.0134 - val_accuracy: 0.4836
Epoch 2/100
317/317 [==============================] - 1s 4ms/step - loss: 1.0582 - accuracy: 0.4794 - val_loss: 0.8745 - val_accuracy: 0.4988
Epoch 3/100
317/317 [==============================] - 2s 6ms/step - loss: 0.9463 - accuracy: 0.4924 - val_loss: 0.8210 - val_accuracy: 0.5138
Epoch 4/100
317/317 [==============================] - 2s 7ms/step - loss: 0.8880 - accuracy: 0.5105 - val_loss: 0.7977 - val_accuracy: 0.5320
Epoch 5/100
317/317 [==============================] - 1s 4ms/step - loss: 0.8523 - accuracy: 0.5210 - val_loss: 0.7847 - val_accuracy: 0.5440
Epoch 6/100
317/317 [==============================] - 1s 4ms/step - loss: 0.8351 - accuracy: 0.5307 - val_loss: 0.7771 - val_accuracy: 0.5509
Epoch 7/100
317/317 [==============================] - 1s 4ms/step - loss: 0.8226 - accuracy: 0.5241 - val_los

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['my_scaler.pkl']